In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Считаем данные и подключим библиотеки

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
df_train = pd.read_csv("../input/tmdb-box-office-prediction/train.csv")
df_test = pd.read_csv("../input/tmdb-box-office-prediction/test.csv")

Посмотрим на наши данные

In [ ]:
df_train.shape 

In [ ]:
df_train.head(3)

Очень много полей с описанием.  

In [ ]:
df_train.info()

Целевая переменная Выручка - Revenue

Признаки belong_to_collection,Home_page слабо представлены 604 и 906 скорее всего они не заполнены как следует. Удалим их позже. 

Почти все данные это object (((

In [ ]:
df_train.describe()

Очень большой разброс от мин до макс и средней - Бюджет, Популярность, Время показа

In [ ]:
df_train['cast'][0] 

In [ ]:
df_train["cast"].fillna("[]", inplace=True) #Заменим NAN на пустые строки.  

In [ ]:
df_train["cast"] = df_train["cast"].apply(eval) #Спер где то на форуме предобразовывает данные
# из str в list. Иначе не работает. 

In [ ]:
#Вытащим ID актеров из этого странного формата  
all_actors = []
for movie_cast in df_train["cast"]:
    for member in movie_cast:
        all_actors.append(member["id"])
all_actors = pd.Series(all_actors)
popular_actors = all_actors.value_counts()[:250].index.values
popular_actors

In [ ]:
#Сделаем тоже самое с другими полями 
list_features = ["cast", "genres", "production_companies", "production_countries", 
                 "spoken_languages", "Keywords", "crew"]
for x in list_features[1:]:
    df_train[x].fillna("[]", inplace=True) 
    df_train[x] = df_train[x].apply(eval) #Переводим в списки

Создаем списки для других столбоцв

In [ ]:
all_directors = [] 
for movie_crew in df_train["crew"]:
    for member in movie_crew:
        if member["job"] == "Director":
            all_directors.append(member["id"])
all_directors = pd.Series(all_directors)
popular_directors = all_directors.value_counts()[:250].index.values
popular_directors

In [ ]:
all_prod_companies = []
for movie in df_train["production_companies"]:
    for company in movie:
        all_prod_companies.append(company["id"])
all_prod_companies = pd.Series(all_prod_companies)
popular_companies = all_prod_companies.value_counts()[:10].index.values
popular_companies

In [ ]:
all_prod_countries = []
for movie in df_train["production_countries"]:
    for country in movie:
        all_prod_countries.append(country["iso_3166_1"])
all_prod_countries = pd.Series(all_prod_countries)
popular_countries = all_prod_countries.value_counts()[:5].index.values
popular_countries

In [ ]:
all_genres = []
for genre_list in df_train["genres"]: 
    for genre in genre_list:
        all_genres.append(genre["name"])
all_genres = pd.Series(all_genres)
popular_genres = np.append(all_genres.value_counts()[:10].index.values, "Animation")
popular_genres

In [ ]:
def count_popular_actors(movie_cast):
    number_of_popular_actors = 0 
    for member in movie_cast:
        if member["id"] in popular_actors: number_of_popular_actors += 1 
    return number_of_popular_actors if number_of_popular_actors < 8 else 8 #movies with number of popular actors > 8 are strange in this dataset
def check_has_popular_director(movie_crew):
    for member in movie_crew: 
        if member["job"] == "Director" and member["id"] in popular_directors: return True     
    return False #if we iterated through all members and haven't found any popular directors
def check_has_popular_company(movie_companies):
    for company in movie_companies: 
        if company["id"] in popular_companies: return True
    return False
def check_has_popular_country(movie_countries):
    for country in movie_countries: 
        if country["iso_3166_1"] in popular_countries: return True   
    return False
def check_genre(movie_genres, target_genre): 
    for genre in movie_genres:
        if target_genre == genre["name"]: return True
    return False

df_train["has_popular_director"] = df_train["crew"].apply(check_has_popular_director)
df_train["number_of_popular_actors"] = df_train["cast"].apply(count_popular_actors)
df_train["from_popular_company"] = df_train["production_companies"].apply(check_has_popular_company)
df_train["from_popular_country"] = df_train["production_countries"].apply(check_has_popular_country)
for genre in popular_genres: df_train["Is"+"".join(genre.split(" "))] = df_train["genres"].apply(check_genre, target_genre=genre)
#print(data.groupby("has_popular_director")["revenue"].mean())
#print(data.groupby("number_of_popular_actors")["revenue"].mean())
#print(data.groupby("from_popular_company")["revenue"].mean())
#print(data.groupby("from_popular_country")["revenue"].mean())
#for genre in popular_genres: print(data.groupby("Is"+"".join(genre.split(" ")))["revenue"].mean())

In [ ]:
df_train.info()

In [ ]:
df_train.drop(["belongs_to_collection","homepage", "cast", "crew", "title", "tagline", "spoken_languages", "production_companies", "Keywords", "poster_path", "status",
           "production_countries", "overview", "original_title", "original_language", "imdb_id", "genres"], axis=1, inplace=True)

Преборазуем дату выпуска на год релиза и сезон 

In [ ]:
def get_year(release_date):
    century = "20" if release_date[-2] in ["0", "1"] else "19"
    return int(century+release_date[-2:])
def get_season(release_date):
    int_month = int(release_date.split("/")[0])
    if int_month < 3: return "Winter"
    if int_month < 6: return "Spring"
    if int_month < 9: return "Summer"
    if int_month < 12: return "Autumn"
    return "Christmas" #yeah, I know, very logical..
df_train["release_year"] = df_train["release_date"].apply(get_year)
df_train["release_season"] = df_train["release_date"].apply(get_season)

In [ ]:
df_train["ReleaseWinter"] = df_train["release_season"] == "Winter"
df_train["ReleaseSpring"] = df_train["release_season"] == "Spring" 
df_train["ReleaseSummer"] = df_train["release_season"] == "Summer" 
df_train["ReleaseAutumn" ] = df_train["release_season"] == "Autumn" 
df_train["ReleaseChristmas"] = df_train["release_season"] == "Christmas" 

In [ ]:
df_train.drop("release_season", axis=1, inplace=True) 

Посморим как растут доходы от фильмов по годам. 

In [ ]:
plt.scatter(df_train["release_year"], df_train["revenue"])

Обычно саммые касоовые новинки выходят под конец года к Рождесту и НГ , проверим на графике

In [ ]:
plt.scatter(df_train["release_season"], df_train["revenue"])

In [ ]:
#for key in df_train['production_companies'][0]['name']:
#    print(key, df_train['production_companies'][0]['name'][key])

In [ ]:
#Удаляем мало представленные признаки.
#df_train.drop('belongs_to_collection', axis=1, inplace=True)
#df_train.drop('homepage', axis=1, inplace=True)

#df_test.drop('belongs_to_collection', axis=1, inplace=True)
#df_test.drop('homepage', axis=1, inplace=True)
